#Part A

In [1]:
!pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp310-cp310-linux_x86_64.whl size=3365683 sha256=68c02a876e113ddaee7dd9b16fcf73716d8b0a1c777a62d4fc645f2905c0e89b
  Stored in directory: /root/.cache/pip/wheels/e8/1b/b5/97ec4cfccdde26e0f3590ad6e09a5242d508dff09704ef86c1
Successfully built mpi4py


## Question 1

In [2]:
%%writefile question-1-mpi.py

import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)
receivebuffer = numpy.zeros(1)
randNum[0] = numpy.random.random_sample(1)

if rank == 1:
    print("Process 1 before receiving has the number", randNum[0])
    comm.Recv(receivebuffer, source=0)
    print("Process 1 received the number", receivebuffer[0])
    bigger = max(randNum[0], receivebuffer[0])
    print("Process 1 has max value ", bigger)
    comm.Send(bigger, dest=0)

if rank == 0:
    print("Process 0 drew the number", randNum[0])

    comm.Send(randNum, dest=1)
    comm.Recv(receivebuffer, source=1)
    print("Process 0 has max value ", receivebuffer[0])



Writing question-1-mpi.py


For Q1, we expect the following cell output to be of the form:


```
Process 0 drew the number 0.0177029648837449
Process 1 before receiving has the number 0.1308715233582639
Process 1 received the number 0.0177029648837449
Process 1 has max value 0.1308715233582639
Process 0 has max value 0.1308715233582639
```


(but with different numbers).

In [3]:
!mpiexec --allow-run-as-root --oversubscribe -n 2 python question-1-mpi.py

Process 0 drew the number 0.6954409135449617
Process 1 before receiving has the number 0.36124728405715756
Process 1 received the number 0.6954409135449617
Process 1 has max value  0.6954409135449617
Process 0 has max value  0.6954409135449617


## Question 2

In [4]:
%%writefile question-2-mpi.py

import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)
receivebuffer = numpy.zeros(1)

if rank == 1:
    print("Process 1 before receiving has the number", randNum[0])
    
    randNum[0] = numpy.random.random_sample(1)
    print("Process 1 after generating sample", randNum[0])
    comm.Irecv(receivebuffer, source=0).wait()
    print("Process 1 received the number", receivebuffer[0])

    bigger = max(randNum[0], receivebuffer[0])
    print("Process 1 has max value ", bigger)

    comm.Isend(bigger, dest=0)

if rank == 0:
    randNum[0] = numpy.random.random_sample(1)
    print("Process 0 drew the number", randNum[0])

    comm.Isend(randNum, dest=1)

    comm.Irecv(receivebuffer, source=1).wait()
    
    print("Process 0 has max value ", receivebuffer[0])


Writing question-2-mpi.py


For Q2, we expect the following cell output to be in the form:

```
Process 1 before receiving has the number 0.0
Process 0 drew the number 0.714475096728165
Process 1 after generating sample 0.5177927276591507
Process 1 received the number 0.714475096728165
Process 1 has max value 0.714475096728165
Process 0 has max value 0.714475096728165
```
(but, again, with different numbers).


In [5]:
!mpiexec --allow-run-as-root --oversubscribe -n 2 python question-2-mpi.py

Process 1 before receiving has the number 0.0
Process 1 after generating sample 0.41413092935852913
Process 0 drew the number 0.24738626081378612
Process 1 received the number 0.24738626081378612
Process 1 has max value  0.41413092935852913
Process 0 has max value  0.41413092935852913


##Question 3

In [6]:
%%writefile question-3-mpi.py

import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()


randNum = numpy.zeros(1)
randNum[0] = numpy.random.random_sample(1)
print("Process", rank, "drew the number", randNum[0])

biggest_num = numpy.zeros(1)
comm.Reduce(randNum, biggest_num, MPI.MAX, root=0)

if rank == 0:
  print("Process 0 has max value", biggest_num[0])


Writing question-3-mpi.py


For Q3 (testing with 8 processes), we expect the following cell output to be in the form:

```
Process 3 drew the number 0.644414619435742
Process 7 drew the number 0.3318780441129008
Process 1 drew the number 0.30690199619345326
Process 2 drew the number 0.3000357589861674
Process 0 drew the number 0.37527495488187257
Process 4 drew the number 0.5049044705082459
Process 5 drew the number 0.748826873661416
Process 6 drew the number 0.44233981192412875
Process 0 has max value 0.748826873661416
```

(but with different numbers).



In [7]:
!mpiexec --allow-run-as-root --oversubscribe -n 8 python question-3-mpi.py

Process 2 drew the number 0.9245021132194493
Process 4 drew the number 0.8818087550614695
Process 6 drew the number 0.4745548146077062
Process 7 drew the number 0.537542247783074
Process 1 drew the number 0.6301321746331191
Process 3 drew the number 0.1070882009157611
Process 5 drew the number 0.06609820483048212
Process 0 drew the number 0.11515836166450777
Process 0 has max value 0.9245021132194493


##Question 4

In [8]:
%%writefile question-4-mpi.py

import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

## YOUR CODE HERE
status = MPI.Status()
randNum = numpy.zeros(1)
randNum[0] = numpy.random.random_sample(1)
print("Process", rank, "drew the number", randNum[0])

if rank == 0:
    size = comm.Get_size()
    biggestNum = randNum  # Reuse this memory
    biggestSrc = 0
    incomingNum = numpy.zeros(1)
    for _ in range(1, size): # we ignore ourselves
        comm.Recv(incomingNum, source=MPI.ANY_SOURCE, status=status)
        if incomingNum[0] > biggestNum[0]:
            biggestNum[0] = incomingNum[0]
            biggestSrc = status.source
    print("Process 0 has max value", biggestNum[0], "found at rank", biggestSrc)
else:
    comm.Send(randNum, dest=0)



Writing question-4-mpi.py


For Q4, we expect the following cell output to be in the form:


```
Process 0 drew the number 0.11288481815163798
Process 4 drew the number 0.060721653476019655
Process 3 drew the number 0.13311050208290287
Process 6 drew the number 0.6279276536062464
Process 5 drew the number 0.7787862146779945
Process 1 drew the number 0.17355707061903436
Process 2 drew the number 0.3926657952242526
Process 7 drew the number 0.3321183350590743
Process 0 max random number 0.7787862146779945 found at rank 5

```
(but with different numbers)


In [9]:
!mpiexec --allow-run-as-root --oversubscribe -n 8 python question-4-mpi.py

Process 1 drew the number 0.7469798203733721
Process 5 drew the number 0.3537527017320806
Process 7 drew the number 0.5703982293721145
Process 2 drew the number 0.6355981660123224
Process 0 drew the number 0.2610898513192419
Process 4 drew the number 0.8242023603834463
Process 6 drew the number 0.9522024035682978
Process 3 drew the number 0.22808189768112908
Process 0 has max value 0.9522024035682978 found at rank 6


#Part B

##Question 1

before jit

In [10]:
import numpy as np
import numba
from numba import jit

def extract_bit(n, i):
  return n >> i & 1 == 1

print(extract_bit(0b1101, 0))
print(extract_bit(0b1101, 1))
print(extract_bit(0b1101, 2))
print(extract_bit(0b1101, 3))


True
False
True
True


In [11]:
def check_circuit(start, end):
  v = np.ones(32, dtype=np.int32)
  count = 0
  for bits in range(start, end):
    for i in range(32):
      v[i] = extract_bit(bits, i)

    if (((v[0] or v[1]) and (not v[1] or not v[3]) and (v[2] or v[3])
       and (not v[3] or not v[4]) and (v[4] or not v[5])
       and (v[5] or not v[6]) and (v[5] or v[6])
       and (v[6] or not v[15]) and (v[7] or not v[8])
       and (not v[7] or not v[13]) and (v[8] or v[9])
       and (v[8] or not v[9]) and (not v[9] or not v[10])
       and (v[9] or v[11]) and (v[10] or v[11])
       and (v[12] or v[13]) and (v[13] or not v[14])
       and (v[14] or v[15]))
       and
          ( (v[16] or v[17]) and (not v[17] or not v[19]) and (v[18] or v[19])
       and (not v[19] or not v[20]) and (v[20] or not v[21])
       and (v[21] or not v[22]) and (v[21] or v[22])
       and (v[22] or not v[31]) and (v[23] or not v[24])
       and (not v[23] or not v[29]) and (v[24] or v[25])
       and (v[24] or not v[25]) and (not v[25] or not v[26])
       and (v[25] or v[27]) and (v[26] or v[27])
       and (v[28] or v[29]) and (v[29] or not v[30])
       and (v[30] or v[31]))):
      count += 1
  return count


In [12]:
%time print(check_circuit(0, 1048576))


0
CPU times: user 10.8 s, sys: 26.3 ms, total: 10.8 s
Wall time: 11.8 s


after jit

In [16]:
from numba import jit

@jit
def extract_bit_jit(n, i):
  return n >> i & 1 == 1

@jit
def check_circuit_jit(start, end):
  v = np.ones(32, dtype=np.int32)
  count = 0
  for bits in range(start, end):
    for i in range(32):
      # Calling the extract_bit function was causing warnings & causing massive slowdown
      # v[i] = extract_bit_jit(bits, i)
      v[i] = (bits >> i & 1 == 1)

    if (((v[0] or v[1]) and (not v[1] or not v[3]) and (v[2] or v[3])
       and (not v[3] or not v[4]) and (v[4] or not v[5])
       and (v[5] or not v[6]) and (v[5] or v[6])
       and (v[6] or not v[15]) and (v[7] or not v[8])
       and (not v[7] or not v[13]) and (v[8] or v[9])
       and (v[8] or not v[9]) and (not v[9] or not v[10])
       and (v[9] or v[11]) and (v[10] or v[11])
       and (v[12] or v[13]) and (v[13] or not v[14])
       and (v[14] or v[15]))
       and
          ( (v[16] or v[17]) and (not v[17] or not v[19]) and (v[18] or v[19])
       and (not v[19] or not v[20]) and (v[20] or not v[21])
       and (v[21] or not v[22]) and (v[21] or v[22])
       and (v[22] or not v[31]) and (v[23] or not v[24])
       and (not v[23] or not v[29]) and (v[24] or v[25])
       and (v[24] or not v[25]) and (not v[25] or not v[26])
       and (v[25] or v[27]) and (v[26] or v[27])
       and (v[28] or v[29]) and (v[29] or not v[30])
       and (v[30] or v[31]))):
      count += 1
  return count

In [19]:
%time print(check_circuit_jit(0, 1048576)) # notice the function name change here
%time print(check_circuit_jit(0, 4294967296)) # full 32-bit test.


0
CPU times: user 35 ms, sys: 0 ns, total: 35 ms
Wall time: 36.5 ms
81
CPU times: user 2min 9s, sys: 398 ms, total: 2min 10s
Wall time: 2min 11s


##Question 2

In [26]:
import os
os.environ['NUMBAPRO_NVVM']='/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so'  
os.environ['NUMBAPRO_LIBDEVICE']='/usr/local/cuda-10.0/nvvm/libdevice'

import numpy as np
import numba
from numba import cuda, jit

@jit
def extract_bit_jit(n, i):
    return n >> i & 1 == 1

@cuda.jit
def check_circuit_gpu(start, end, count_out):
  id = cuda.grid(1)
  per = (end - start) // cuda.gridsize(1)
  thread_start = id * per
  thread_end = (id+1) * per

  v = cuda.local.array(32, dtype=numba.int32)
  count = 0
  for bits in range(thread_start, thread_end):
    for i in range(32):
        # Calling the extract_bit function was causing warnings
        v[i] = (bits >> i & 1 == 1)

        if (((v[0] or v[1]) and (not v[1] or not v[3]) and (v[2] or v[3])
          and (not v[3] or not v[4]) and (v[4] or not v[5])
          and (v[5] or not v[6]) and (v[5] or v[6])
          and (v[6] or not v[15]) and (v[7] or not v[8])
          and (not v[7] or not v[13]) and (v[8] or v[9])
          and (v[8] or not v[9]) and (not v[9] or not v[10])
          and (v[9] or v[11]) and (v[10] or v[11])
          and (v[12] or v[13]) and (v[13] or not v[14])
          and (v[14] or v[15]))
          and
              ( (v[16] or v[17]) and (not v[17] or not v[19]) and (v[18] or v[19])
          and (not v[19] or not v[20]) and (v[20] or not v[21])
          and (v[21] or not v[22]) and (v[21] or v[22])
          and (v[22] or not v[31]) and (v[23] or not v[24])
          and (not v[23] or not v[29]) and (v[24] or v[25])
          and (v[24] or not v[25]) and (not v[25] or not v[26])
          and (v[25] or v[27]) and (v[26] or v[27])
          and (v[28] or v[29]) and (v[29] or not v[30])
          and (v[30] or v[31]))):
            count += 1
  cuda.atomic.add(count_out, 0, count)

In [27]:
# part b
#16384 threads should take around 52 ms, 1 will take ~5 minutes
count_out = cuda.to_device(np.zeros(shape=1, dtype=np.int32))
check_circuit_gpu[128, 128](0, 4294967296, count_out)
%time print(count_out.copy_to_host(), 'solutions')

[2592] solutions
CPU times: user 2.8 s, sys: 3.01 ms, total: 2.8 s
Wall time: 2.8 s


In [28]:
#part c
## YOUR CODE HERE
for n in [1,8,16,32,64,128,256,512,1024,2048,4096]:
  count_out = cuda.to_device(np.zeros(shape=1, dtype=np.int32))
  check_circuit_gpu[n, 128](0, 4294967296, count_out)
  print(f"[{n}, 128:]")
  %time print(count_out.copy_to_host(), 'solutions')
  print()

/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


[1, 128:]
[2592] solutions
CPU times: user 2min 7s, sys: 107 ms, total: 2min 7s
Wall time: 2min 8s

[8, 128:]


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 8 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


[2592] solutions
CPU times: user 15.9 s, sys: 14 ms, total: 15.9 s
Wall time: 15.9 s

[16, 128:]


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 16 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


[2592] solutions
CPU times: user 8 s, sys: 6 ms, total: 8.01 s
Wall time: 7.98 s

[32, 128:]


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 32 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


[2592] solutions
CPU times: user 3.98 s, sys: 5.97 ms, total: 3.99 s
Wall time: 4 s

[64, 128:]


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 64 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


[2592] solutions
CPU times: user 2.92 s, sys: 1.99 ms, total: 2.92 s
Wall time: 2.91 s

[128, 128:]
[2592] solutions
CPU times: user 2.76 s, sys: 3.01 ms, total: 2.76 s
Wall time: 2.75 s

[256, 128:]
[2592] solutions
CPU times: user 2.38 s, sys: 993 µs, total: 2.38 s
Wall time: 2.37 s

[512, 128:]
[2592] solutions
CPU times: user 2.16 s, sys: 2 ms, total: 2.16 s
Wall time: 2.16 s

[1024, 128:]
[2592] solutions
CPU times: user 2.12 s, sys: 0 ns, total: 2.12 s
Wall time: 2.14 s

[2048, 128:]
[2592] solutions
CPU times: user 2.13 s, sys: 1.96 ms, total: 2.13 s
Wall time: 2.13 s

[4096, 128:]
[2592] solutions
CPU times: user 2.11 s, sys: 1.01 ms, total: 2.11 s
Wall time: 2.1 s



##Question 3

In [34]:
import numpy as np
import numba
from numba import cuda, jit
import os
os.environ['NUMBAPRO_NVVM']='/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so'  
os.environ['NUMBAPRO_LIBDEVICE']='/usr/local/cuda-10.0/nvvm/libdevice'

@jit
def median(a, b, c):
  if a > b:
    if b > c:
      return b 
    elif a > c: 
      return c 
    else:
      return a  
  else: 
    if a > c:
      return a
    elif b > c:
      return c
    else:
      return b 

@jit
def median_filter_seq(A):
  n = np.shape(A)[0]
  r = np.zeros(n, dtype=np.uint8)
  r[0] = A[0]
  r[n-1] = A[n-1]
  
  for i in range(1,n-1):
    r[i] = median(A[i-1], A[i], A[i+1])
  
  return r

A = np.random.randint(256, size=256, dtype=np.uint8)
B = np.array([1, 5, 3, 4, 2, 9, 4])

A_seq = median_filter_seq(A)
B_seq = median_filter_seq(B)

@cuda.jit
def median_filter(A):
  i = cuda.grid(1)
  ## YOUR CODE HERE
  if i == 0 or i == len(A) - 1:
    return
  prev_val = A[i-1]
  current_val = A[i]
  next_val = A[i+1]
  A[i] = median(prev_val, current_val, next_val)



median_filter[1,256](A)
median_filter[1,7](B)

assert np.array_equal(B, B_seq)
assert np.array_equal(A, A_seq)

print("Assertions passed")


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Assertions passed


/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
